In [ ]:
import os
import sys
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import rasterio as rio
from rasterio.plot import show
from rasterio.merge import merge
from pathlib import Path
import masking as msk

### When to run this notebook

PARGE does not output files with proper nodata masks outside the flightline area. This notebook creates boolean raster masks according to your specifications. Please adapt based on what input files you want to use (radiance, reflectance, scan angle?) and whether you have separate VNIR/SWIR rasters or a combined supercube from integrated processing. 

Raster masks can then be converted to vector masks with GDAL, such as:

` gdal_polygonize.py -8 rastermasks/vnir_swir_05_mask.bsq vectormasks_flightline/vnir_swir_05_mask.shp` 

In [ ]:
inputdir = Path(r"Z:/fihyper/cwaigl/20190817_CPC/20190817-CPC_03/Parge_with_offsets/")
outputdir = Path(r"Z:/fihyper/cwaigl/20190817_CPC/02_intermediate/")

In [ ]:
fnvnir = "20210803-BC_02_VNIR_SWIR_rad_geo.bsq"
fnswir = "20210803-BC_02_VNIR_SWIR_rad_geo.bsq"
fnvnir_sca = "20210803-BC_02_VNIR_SWIR_rad_geo_sca.bsq"
fnswir_sca = "20210803-BC_02_VNIR_SWIR_rad_geo_sca.bsq"

In [ ]:
os.chdir(inputdir)

### Helper functions

Find start and end of valid data, presuming a lower bound nodata value. Optionally switch to upper bound (set `reversesign` to `True`.) 

In [ ]:
def getdatabounds(row, nodataval=0.0, reversesign=False):
    prefac = -1 if reversesign else 1
    cond = prefac * row > prefac * nodataval
    return np.nonzero(cond)[0][0], np.nonzero(cond)[0][-1]

Crop a row by setting all values outside the left and right bounds to the desired nodata value (0 by default). Optionnally reduce row by a fixed proportion. 

In [ ]:
def croprow(row, lprop=0.0, rprop=0.0, nodataval=0):
    try:
        lidx, ridx = getdatabounds(row)
        lidx = lidx + int((ridx - lidx) * lprop)
        ridx = ridx - int((ridx - lidx) * rprop)
        row[:lidx] = nodataval
        row[ridx:] = nodataval
    except IndexError:
        pass
    return row

In [ ]:
def maskrow(row, lprop=0.0, rprop=0.0, nodataval=0):
    newrow = np.ones(len(row))
    try:
        lbound, rbound = getdatabounds(row)
        newrow[:lbound] = nodataval
        newrow[rbound+1:] = nodataval
        return newrow
    except IndexError:
        return nodataval * newrow

def maskrow_l(row, lprop=0.1, rprop=0.0, nodataval=0):
    newrow = np.ones(len(row))
    try:
        lidx, ridx = getdatabounds(row)
        lidx = lidx + int((ridx - lidx) * lprop)
        ridx = ridx - int((ridx - lidx) * rprop)
        newrow[:lidx] = nodataval
        newrow[ridx+1:] = nodataval
        return newrow
    except IndexError:
        return nodataval * newrow
    
def maskrow_r(row, lprop=0.0, rprop=0.1, nodataval=0):
    newrow = np.ones(len(row))
    try:
        lidx, ridx = getdatabounds(row)
        lidx = lidx + int((ridx - lidx) * lprop)
        ridx = ridx - int((ridx - lidx) * rprop)
        newrow[:lidx] = nodataval
        newrow[ridx+1:] = nodataval
        return newrow
    except IndexError:
        return nodataval * newrow

In [ ]:
maskrow_r(np.array([0,0,0, 0,1,2,3,4,1, 1, 1, 1, 1, 1, 1, 1, 5,6,7,8,9, 1, 0]))

array([0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 0.])

### Create 1 nodata mask from separately processed VNIR/SWIR radiance files and scanangle files

In [ ]:
with rio.open(fnvnir) as vnir:
    with rio.open(fnswir) as swir:
        with rio.open(fnvnir_sca) as vnir_sca:
            with rio.open(fnswir_sca) as swir_sca:
                outmeta = vnir.meta.copy()
                vnir_sampleband = vnir.read(1)
                print("read vnir band")
                swir_sampleband = swir.read(200)
                print("read swir band")
                vnir_alt = vnir_sca.read(3)
                print("read vnir altitude")
                swir_alt = swir_sca.read(3)
                print("read swir altitude")
                maskvnir = np.apply_along_axis(maskrow, 1, vnir_sampleband)
                print("created vnir mask")
                maskswir = np.apply_along_axis(maskrow, 1, swir_sampleband)
                print("created swir mask")
                maskvnir_alt = np.apply_along_axis(maskrow, 1, vnir_alt)
                print("created vnir altitude mask")
                maskswir_alt = np.apply_along_axis(maskrow, 1, swir_alt)
                print("created swir altitude mask")
                outdata = maskvnir * maskswir * maskvnir_alt * maskswir_alt


read vnir band
read swir band
read vnir altitude
read swir altitude
created vnir mask
created swir mask
created vnir altitude mask
created swir altitude mask


In [ ]:
outdata = outdata.astype('int16')

In [ ]:
outmeta.update({
    'dtype': 'int16',
    'count': 1,
    'nodata': 0
})

In [ ]:
outfn = os.path.join(outputdir, 'vnir_swir_02_mask.bsq')
with rio.open(outfn, "w", **outmeta) as dst:
    dst.write(outdata, indexes=1)

This code was transfered to `masking.py` 

### Looping and mask creation for VNIR/SWIR stacks from integrated processing

In [ ]:
import json

In [ ]:
projdir = Path(r"Z:\fihyper\cwaigl\20200830_BC")
prefix = "20200830-BC"
subdir = "Parge_offsets_integrated"
jsonfile_suffix = "_VNIR_SWIR_rad_geo_flightdata.txt"
linedirs = list(projdir.glob(f"{prefix}_*"))
outputdir = projdir / "02_intermediate/rastermasks"

In [ ]:
for linedir in linedirs[4:5]:
    print(f"working in {linedir}")
    with (linedir / "NAV" / f"{linedir.parts[-1]}{jsonfile_suffix}").open() as src:
        meta = json.load(src)
    fn = f"{linedir.parts[-1]}_VNIR_SWIR_rad_geo.bsq"
    fn_sca = f"{linedir.parts[-1]}_VNIR_SWIR_rad_geo_sca.bsq"
    lineno = linedir.parts[-1][-2:]
    if meta['heading_avg'] < 20 or meta['heading_avg'] > 340:
        maskfunc = maskrow_r
    else:
        maskfunc = maskrow_l
    with rio.open(linedir / subdir / fn) as data:
        with rio.open(linedir / subdir / fn_sca) as data_sca:
            outmeta = data.meta.copy()
            vnir_sampleband = data.read(1)
            print("read vnir band")
            swir_sampleband = data.read(200)
            print("read swir band")
            altband = data_sca.read(3)
            print("read  altitude")
            maskvnir = np.apply_along_axis(maskfunc, 1, vnir_sampleband)
            print("created vnir mask")
            maskswir = np.apply_along_axis(maskfunc, 1, swir_sampleband)
            print("created swir mask")
            mask_alt = np.apply_along_axis(maskfunc, 1, altband)
            print("created altitude mask")
            outdata = maskvnir * maskswir * mask_alt 
    outdata = outdata.astype('int16')
    outmeta.update({
        'dtype': 'int16',
        'count': 1,
        'nodata': 0
    })
    outfn = outputdir / f'vnir_swir_{lineno}_mask.bsq'
    with rio.open(outfn, "w", **outmeta) as dst:
        print(f"writing {outfn}")
        dst.write(outdata, indexes=1)

working in Z:\fihyper\cwaigl\20200830_BC\20200830-BC_05
read vnir band
read swir band
read  altitude
created vnir mask
created swir mask
created altitude mask
writing Z:\fihyper\cwaigl\20200830_BC\02_intermediate\rastermasks\vnir_swir_05_mask.bsq


In [ ]:
linedirs

[WindowsPath('Z:/fihyper/cwaigl/20200830_BC/20200830-BC_01'),
 WindowsPath('Z:/fihyper/cwaigl/20200830_BC/20200830-BC_02'),
 WindowsPath('Z:/fihyper/cwaigl/20200830_BC/20200830-BC_03'),
 WindowsPath('Z:/fihyper/cwaigl/20200830_BC/20200830-BC_04'),
 WindowsPath('Z:/fihyper/cwaigl/20200830_BC/20200830-BC_05'),
 WindowsPath('Z:/fihyper/cwaigl/20200830_BC/20200830-BC_06'),
 WindowsPath('Z:/fihyper/cwaigl/20200830_BC/20200830-BC_07')]

###  Change to a new data project directory

In [ ]:
os.chdir("Z:/fihyper/cwaigl/20190817_CPC/")

In [ ]:
os.listdir()

['20190817-CPC_01',
 '20190817-CPC_02',
 '20190817-CPC_03',
 '20190817-CPC_04',
 '20190817-CPC_05',
 '20190817-CPC_06',
 '20190817-CPC_07',
 '20190817-CPC_08',
 '20190817-CPC_09',
 '.DS_Store',
 '20190817-CPC_10',
 '._.DS_Store',
 '01_inputs',
 '20190817-CPC_11',
 '00_aux',
 'x0_tmp',
 '02_intermediate',
 '03_products',
 'xx_misc_orig_processing_files',
 '20190817-CPC_12',
 '20190817-CPC_13',
 '20190817-CPC_14',
 '20190817-CPC_15',
 '20190817-CPC_16',
 '20190817-CPC_17',
 '20190817-CPC_18',
 '20190817-CPC_19',
 '20190817-CPC_20',
 '20190817-CPC_21',
 '20190817-CPC_22',
 '20190817-CPC_23',
 '20190817-CPC_24',
 '20190817-CPC_25',
 '20190817-CPC_26']

In [ ]:
dirprefix = '20190817-CPC'
flightlines = [nm for nm in os.listdir() if nm.startswith(dirprefix)]
flightlines

['20190817-CPC_01',
 '20190817-CPC_02',
 '20190817-CPC_03',
 '20190817-CPC_04',
 '20190817-CPC_05',
 '20190817-CPC_06',
 '20190817-CPC_07',
 '20190817-CPC_08',
 '20190817-CPC_09',
 '20190817-CPC_10',
 '20190817-CPC_11',
 '20190817-CPC_12',
 '20190817-CPC_13',
 '20190817-CPC_14',
 '20190817-CPC_15',
 '20190817-CPC_16',
 '20190817-CPC_17',
 '20190817-CPC_18',
 '20190817-CPC_19',
 '20190817-CPC_20',
 '20190817-CPC_21',
 '20190817-CPC_22',
 '20190817-CPC_23',
 '20190817-CPC_24',
 '20190817-CPC_25',
 '20190817-CPC_26']

Sometimes the output files have an extra `_rad` element.

In [ ]:
extra = '_rad'
#extra = ''

**Optional:** Only run the following cell if you want to pick out certain flightlines only

In [ ]:
selected = [flightlines[ii] for ii in [1, 2, 3, 6]]
flightlines = selected

### Loop through flightlines and create masks for separately prepared VNIR, SWIR and scanagnle files

In [ ]:
for linedir in flightlines[-1:]:
    linenumstr = linedir[-2:]
    inputdir_vnir = f"{linedir}\\Parge_with_offsets"
    inputdir_swir = inputdir_vnir
    outputdir = f"{linedir}\\ATCOR"
    Path(outputdir).mkdir(parents=True, exist_ok=True)
    fnvnir = os.path.join(inputdir_vnir, f"{dirprefix}_{linenumstr}_VNIR_1800_SN00812_FOVx2_raw{extra}_bsq_float32_geo.bsq")
    fnswir = os.path.join(inputdir_swir, f"{dirprefix}_{linenumstr}_SWIR_384me_SN3107_FOVx2_raw{extra}_bsq_float32_geo.bsq")
    fnvnir_sca = os.path.join(inputdir_vnir, f"{dirprefix}_{linenumstr}_VNIR_1800_SN00812_FOVx2_raw{extra}_bsq_float32_geo_sca.bsq")
    fnswir_sca = os.path.join(inputdir_swir, f"{dirprefix}_{linenumstr}_SWIR_384me_SN3107_FOVx2_raw{extra}_bsq_float32_geo_sca.bsq")
    print(os.path.exists(fnvnir), os.path.exists(fnswir), 
          os.path.exists(fnvnir_sca), os.path.exists(fnswir_sca))
    try:
        outdata, outmeta = msk.getflightlinemask(fnvnir, fnswir, fnvnir_sca, fnswir_sca)
    except: 
        print(f"Problem opening files in {linedir}")
        continue
    outfn = os.path.join(outputdir, f'vnir_swir_{linenumstr}_mask.bsq')
    with rio.open(outfn, "w", **outmeta) as dst:
        dst.write(outdata, indexes=1)
        print(f"Wrote vnir_swir_{linenumstr}_mask.bsq to {outputdir}")

True True False False
Didn't find all fines for flightline
Problem opening files in 20190817-CPC_26


### Loop throught flightlines and make masked RGB

In [ ]:
for linedir in flightlines:
    linenumstr = linedir[-2:]
    print(f"Working on line {linenumstr}")
    inputdir_vnir = f"{linedir}\\Parge_with_offsets"
    inputdir_swir = inputdir_vnir
    outputdir = f"{linedir}\\ATCOR"
    Path(outputdir).mkdir(parents=True, exist_ok=True)
    fnvnir = os.path.join(inputdir_vnir, f"{dirprefix}_{linenumstr}_VNIR_1800_SN00812_FOVx2_raw{extra}_bsq_float32_geo.bsq")
    fnswir = os.path.join(inputdir_swir, f"{dirprefix}_{linenumstr}_SWIR_384me_SN3107_FOVx2_raw{extra}_bsq_float32_geo.bsq")

    with rio.open(fnvnir) as vnir:
        outmeta1 = vnir.meta.copy()
        outmeta2 = vnir.meta.copy()
    with rio.open(os.path.join(outputdir, f'vnir_swir_{linenumstr}_mask.bsq')) as maskdata:
        mask = maskdata.read(1)
    outmeta1.update({
        'count': 3,
        'nodata': 0.0
    })
    outmeta2.update({
        'count': 3,
        'driver': 'GTiff',
        'nodata': 0.0
    })    
    outfn1 = os.path.join(inputdir_vnir, f'{dirprefix}_{linenumstr}_vnir_swir_firergb_geo.bsq')
    outfn2 = os.path.join(inputdir_vnir, f'{dirprefix}_{linenumstr}_vnir_swir_firergb_geo.tif')

    with rio.open(outfn1, "w", **outmeta1) as dst1:
        with rio.open(outfn2, "w", **outmeta2) as dst2:
            with rio.open(fnswir) as swir:
                swir_sampleband = swir.read(120)
                bandout = swir_sampleband * mask
                dst1.write(bandout, indexes=1)
                dst2.write(bandout, indexes=1)
            with rio.open(fnvnir) as vnir:
                ii = 2
                for bandnum in [140, 20]:
                    vnir_sampleband = vnir.read(bandnum)
                    bandout = vnir_sampleband * mask
                    dst1.write(bandout, indexes=ii)
                    dst2.write(bandout, indexes=ii)
                    ii = ii + 1

    

In [ ]:
for linedir in flightlines:
    linenumstr = linedir[-2:]
    print(f"Working on line {linenumstr}")
    inputdir_vnir = f"{linedir}\\Parge_with_offsets"
    outputdir = f"{linedir}\\ATCOR"
    Path(outputdir).mkdir(parents=True, exist_ok=True)
    fnvnir = os.path.join(inputdir_vnir, f"{dirprefix}_{linenumstr}_VNIR_1800_SN00812_FOVx2_raw{extra}_bsq_float32_geo.bsq")

    with rio.open(fnvnir) as vnir:
        outmeta1 = vnir.meta.copy()
        outmeta2 = vnir.meta.copy()
    with rio.open(os.path.join(outputdir, f'vnir_swir_{linenumstr}_mask.bsq')) as maskdata:
        mask = maskdata.read(1)
    outmeta1.update({
        'count': 3,
        'nodata': 0.0
    })
    outmeta2.update({
        'count': 3,
        'driver': 'GTiff',
        'nodata': 0.0
    })    
    outfn1 = os.path.join(inputdir_vnir, f'{dirprefix}_{linenumstr}_vnir_landrgb_geo.bsq')
    outfn2 = os.path.join(inputdir_vnir, f'{dirprefix}_{linenumstr}_vnir_landrgb_geo.tif')

    with rio.open(outfn1, "w", **outmeta1) as dst1:
        with rio.open(outfn2, "w", **outmeta2) as dst2:
            with rio.open(fnvnir) as vnir:
                for ii, bandnum in enumerate([75, 45 , 20], 1):
                    vnir_sampleband = vnir.read(bandnum)
                    bandout = vnir_sampleband * mask
                    dst1.write(bandout, indexes=ii)
                    dst2.write(bandout, indexes=ii)

Working on line 01
Working on line 02
Working on line 03
Working on line 04
Working on line 05
Working on line 06
Working on line 07


### Loop through flightlines and stack & mask scan angle files

In [ ]:
for linedir in flightlines[1:-1]:
    linenumstr = linedir[-2:]
    print(f"Working on line {linenumstr}")
    inputdir = f"{linedir}\\Parge_with_offsets"
    outputdir = f"{linedir}\\ATCOR"
    fnvnirsca = os.path.join(inputdir, f"{dirprefix}_{linenumstr}_VNIR_1800_SN00812_FOVx2_raw{extra}_bsq_float32_geo_sca.bsq")
    fnswirsca = os.path.join(inputdir, f"{dirprefix}_{linenumstr}_SWIR_384me_SN3107_FOVx2_raw{extra}_bsq_float32_geo_sca.bsq")
    outfn = os.path.join(outputdir, f'{dirprefix}_{linenumstr}_vnir_swir_supercube_geo_sca.bsq')
    
    try:
        with rio.open(os.path.join(outputdir, f'vnir_swir_{linenumstr}_mask.bsq')) as maskdata:
            mask = maskdata.read(1)
    except rio.RasterioIOError as e:
        print(str(e))
        continue

    with rio.open(fnvnirsca) as vnirsca:
        with rio.open(fnswirsca) as swirsca:
            outmeta = vnirsca.meta.copy()
            outtags = vnirsca.tags().copy()
            with rio.open(outfn, "w", **outmeta) as dst:
                # start with scan angles
                vnirbd = vnirsca.read(1)
                swirbd = swirsca.read(1)
                stackbd = np.rint((vnirbd + swirbd)/2)
                stackbd[mask==0] = 9100
                dst.write(stackbd.astype('int16'), indexes=1)
                # now the other three
                for ii in [2, 3, 4]:
                    vnirbd = vnirsca.read(1)
                    swirbd = swirsca.read(1)
                    stackbd = mask * np.rint((vnirbd + swirbd)/2)
                    dst.write(stackbd.astype('int16'), indexes=ii)
                dst.update_tags(**outtags)

Working on line 02
20190817-CPC_02\ATCOR\vnir_swir_02_mask.bsq: No such file or directory
Working on line 03
Working on line 04
Working on line 05
Working on line 06
Working on line 07
Working on line 08
Working on line 09
Working on line 10
20190817-CPC_10\ATCOR\vnir_swir_10_mask.bsq: No such file or directory
Working on line 11
Working on line 12
20190817-CPC_12\ATCOR\vnir_swir_12_mask.bsq: No such file or directory
Working on line 13
20190817-CPC_13\ATCOR\vnir_swir_13_mask.bsq: No such file or directory
Working on line 14
Working on line 15
Working on line 16
Working on line 17
Working on line 18
Working on line 19
Working on line 20
Working on line 21
Working on line 22
Working on line 23
Working on line 24
Working on line 25
